In [1]:
import cv2
import os
import numpy as np
subjects = ["", "Shreyansh Gupta"]

def detect_face(img):
    #convert the test image to gray image as opencv face detector expects gray images
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')

    #let's detect multiscale (some images may be closer to camera than others) images
    #result is a list of faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5);
    
    #if no faces are detected then return original img
    if (len(faces) == 0):
        return None, None
    
    #under the assumption that there will be only one face,
    #extract the face area
    (x, y, w, h) = faces[0]
    
    #return only the face part of the image
    return gray[y:y+w, x:x+h], faces[0]

def prepare_training_data(data_folder_path):
    
    #------STEP-1--------
    #get the directories in data folder
    dirs = os.listdir(data_folder_path)
    
    #list to hold all subject faces
    faces = []
    #list to hold labels for all subjects
    labels = []
    
    #------STEP-2--------
    #get the images names that are inside the given subject directory
    subject_images_names = os.listdir(data_folder_path)
    
    #This model is trained only on one test case i.e. Shreyansh Gupta
    label = 1
        
    #------STEP-3--------
    #go through each image name, read image, 
    #detect face and add face to list of faces
    for image_name in subject_images_names:
            
        #ignore system files like .DS_Store
        if image_name.startswith("."):
            continue;
            
        #build image path
        #sample image path = training-data/s1/1.pgm
        image_path = data_folder_path + "/" + image_name

        #read image
        image = cv2.imread(image_path)
            
        #display an image window to show the image 
        cv2.imshow("Training on image...", image)
        cv2.waitKey(100)
            
        #detect face
        face, rect = detect_face(image)
            
        #------STEP-4--------
        if face is not None:
            #add face to list of faces
            faces.append(face)
            #add label for this face in this case it is 1
            labels.append(label)
           
    
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    
    return faces, labels

In [2]:
#let's first prepare our training data
#data will be in two lists of same size
#one list will contain all the faces
#and other list will contain respective labels for each face
print("Preparing data...")
faces, labels = prepare_training_data("train")
print("Data prepared")

#print total faces and labels
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))

Preparing data...
Data prepared
Total faces:  32
Total labels:  32


In [3]:
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.train(faces, np.array(labels))

def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
#function to draw text on give image starting from
#passed (x, y) coordinates. 
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)
    
def predict(test_img):
    #make a copy of the image as we don't want to chang original image
    img = test_img.copy()
    #detect face from the image
    face, rect = detect_face(img)

    #predict the image using our face recognizer 
    label= face_recognizer.predict(face)
    #get name of respective label returned by face recognizer
    label_text = subjects[label[0]]
    
    #draw a rectangle around face detected
    draw_rectangle(img, rect)
    #draw name of predicted person
    draw_text(img, label_text, rect[0], rect[1]-5)
    
    return img

print("Predicting images...")

#load test images
test_img1 = cv2.imread("test/1.jpg")
#perform a prediction
predicted_img1 = predict(test_img1)
print("Prediction complete")

#display both images
cv2.imshow(subjects[1], predicted_img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

Predicting images...
Prediction complete


In [ ]:
import dlib, cv2
import socket , zmq
import numpy as np
# We have imoprted cv2 for opening cam and showing images, zmq for communication to server, np for help in transformation
# and dlib for predictions of face and landmarks


context= zmq.Context()
#Here we build the context for communication to server
socket = context.socket(zmq.REP)
#From the context we create a socket through which other clients will be connected 
socket.bind("tcp://127.0.0.1:9999")
#For the example purpose we are taking port 9999 and localhost ip for sake of demonstration
#So far we have set up connection which listens to the ip and binds with it.


# THIS FUNCTION SENDS THE FRAME IMAGE TO THE ENDPOINT
def send_array(socket, A, flags=0, copy=True, track=False):
    md = dict(
        dtype = str(A.dtype),
        shape = A.shape,
    )
    # Made a dictionary of dtype of array and shape of array so that at server side we know
    # at the time of transforming array from  buffer we must know the shape to get it back.

    socket.send_json(md, flags|zmq.SNDMORE)
    #Sent the image using json to the server
    return socket.send(A, flags, copy=copy, track=track)
    # Finally sending the image in form of buffer to the server.

# THIS FUNCTION RECIEVIES THE ARRAY SENT FROM ANOTHER END
def recv_array(socket, flags=0, copy=True, track=False):
    md = socket.recv_json(flags=flags)
    # Recieves the json file containing dtype and shape of the required array
    msg = socket.recv(flags=flags, copy=copy, track=track)
    # msg is the buffer recived which contains the array
    A = np.frombuffer(msg, dtype=md["dtype"])
    # Using numpy and shape known  we transform the buffer into the array and reshape
    # it to required shape and then finally return it.
    return A.reshape(md['shape'])

# The while loop continuously listens to client and revieces frame and 
# send them after processing 
print("Predicting images...")
while True:
    frame = recv_array(socket)

    #perform a prediction
    predicted_img1 = predict(frame)

    #display both images
    cv2.imshow('Prediction on server side', predicted_img1)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    # Now all faces are processed and boxes are drawn

    #Option to quit the process and close the server from inifinte loop by pressing "q"
    k= cv2.waitKey(30) & 0xFF
    if k==ord('q'):
        break
    
    send_array(socket,frame)
    # If the server is still running then we need to send the image back to the client by the socket